In [ ]:
import eikon as ek
from collections import Counter
import pandas as pd
from pandas import DataFrame
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
ek.set_app_key('....')
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

In [ ]:

# dictionary with customised words
import csv


labels = {}

with open(r'.....\dict.csv', mode='r') as inp:
    reader = csv.reader(inp)
    labels = {rows[0]:rows[1] for rows in reader}

print(labels)


In [ ]:
def matcher(k):
    x = (i for i in labels if i in k)
    return ' | '.join(map(labels.get, x))

In [ ]:
df1 = ek.get_news_headlines('Topic:COVID AND Language:LEN AND Source:RTRS', date_from='2021-03-01T09:00:00',date_to='2021-03-02T09:00:00',count=50)
df1['filtered_words']=np.nan
df1['flag_list']=np.nan
df1['counter']=np.nan
for idx, storyId in enumerate(df1['storyId'].values):  #for each row in our df1 dataframe
    newsText = ek.get_news_story(storyId) #get the news story
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentence = newsText.lower()
        sentence=sentence.replace('{html}',"") 
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', sentence)
        rem_url=re.sub(r'http\S+', '',cleantext)
        rem_num = re.sub('[0-9]+', '', rem_url)
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(rem_num)  
        filtered_words = [w for w in tokens if len(w) > 2 ] 
        df1['filtered_words'].iloc[idx] =  filtered_words
        df1['filtered_words'] = df1['filtered_words'].astype(str,'ignore')
        df1['flag_list']=df1['filtered_words'].apply(matcher)
        df1['flag_list'] = df1['flag_list'].astype(str,'ignore')
        df1['number']=df1['filtered_words'].str.count(r'[a-z]')
        df1['number2']=df1['flag_list'].str.count(r'[a-z]')
        df1['fore']=df1['filtered_words'].str.findall(r'\'\b(forebroding|uncertianty|fear|worry)\b\'')
        df1['fore'] = df1['fore'].astype(str,'ignore')
        df1['count_flag']=df1['fore'].str.count(r'[a-z]')
        df1['term_frequency']=df1['count_flag']/df1['number']
        df1['flag_list'] = df1['flag_list'].astype(str,'ignore')
        df1['count_flag_pos']=df1['flag_list'].str.findall(r'(pos)')
        df1['count_flag_pos'] = df1['count_flag_pos'].astype(str,'ignore')
        df1['count_flag_pos1']=df1['count_flag_pos'].str.count(r'[a-z]')
        df1['count_flag_neg']=df1['flag_list'].str.findall(r'neg')
        df1['count_flag_neg'] = df1['count_flag_neg'].astype(str,'ignore')
        df1['count_flag_neg1']=df1['count_flag_neg'].str.count(r'[a-z]')
        df1['sentiment']=(df1['count_flag_pos1']+1)/(df1['count_flag_neg1']+1)
        df1['sentiment2']=(df1['count_flag_pos1'])/(df1['count_flag_neg1']+df1['count_flag_pos1'])
df1.head()